Seting up libreries and data

**Extract features:**

Since SVM classifiers do not learn hierarchical representations from raw EEG signals, explicit feature extraction is required. We therefore employed Common Spatial Patterns (CSP) to project the multi-channel EEG data into a low-dimensional, task-discriminative spatial subspace prior to classification.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install mne

Setup & imports

In [ ]:
import os
import numpy as np
import scipy.io
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_val_score, learning_curve
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, f1_score

try:
    from mne.decoding import CSP
    from mne.filter import filter_data

except ImportError:
    print("Installing MNE...")
    !pip install -q mne
    from mne.decoding import CSP
    from mne.filter import filter_data

Configuration

In [ ]:
# Configuration
subjects = [f"{i:02d}" for i in range(1, 16)]  # ['01', ... '15']
base_path = "/content/drive/MyDrive/FAST-EEG"

# Create directories for saving results if they don't exist
plot_dir = os.path.join(base_path, "Results", "Plots")
model_dir = os.path.join(base_path, "Results", "Models")
os.makedirs(plot_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

results_list = []
print(f"Starting processing for {len(subjects)} subjects...")
print(f"Artifacts will be saved to: {os.path.join(base_path, 'Results')}\n")

Starting processing for 15 subjects...
Artifacts will be saved to: /content/drive/MyDrive/FAST-EEG/Results



Main loop

In [ ]:
# Main Loop across all subjects
for sub in subjects:
    print(f"--- Processing Subject {sub} ---")

    # Step 1: Load Data
    try:
        # Load Training
        f_tr = f"{base_path}/Training_set/Data_Sample{sub}.mat"
        d_tr = scipy.io.loadmat(f_tr)
        x_tr = np.asarray(d_tr["epo_train"]["x"])[0, 0]
        y_tr = np.asarray(d_tr["epo_train"]["y"])[0, 0].argmax(0)
        X_tr = np.transpose(x_tr, (2, 1, 0)).astype(np.float64)

        # Load Validation
        f_va = f"{base_path}/Validation_set/Data_Sample{sub}.mat"
        d_va = scipy.io.loadmat(f_va)
        x_va = np.asarray(d_va["epo_validation"]["x"])[0, 0]
        y_va = np.asarray(d_va["epo_validation"]["y"])[0, 0].argmax(0)
        X_va = np.transpose(x_va, (2, 1, 0)).astype(np.float64)

    except FileNotFoundError:
        print(f"File for Subject {sub} not found. Skipping.")
        continue
    except Exception as e:
        print(f"Error loading Subject {sub}: {e}")
        continue

    # -----------------------------------------------------------------
    # Step 2: Filter the data (4–40 Hz) using bandpass filtering
    # -----------------------------------------------------------------
    sfreq = 250  # sampling rate
    X_tr = filter_data(X_tr, sfreq, l_freq=4, h_freq=40, verbose=False)
    X_va = filter_data(X_va, sfreq, l_freq=4, h_freq=40, verbose=False)

    # -----------------------------------------------------------------
    # Step 3: Define Pipeline
    # -----------------------------------------------------------------
    n_components = 8
    clf = Pipeline([
        ('csp', CSP(n_components=n_components, reg=None, log=True, norm_trace=False)),
        ('scaler', StandardScaler()),
        ('svm', SVC(kernel='rbf', C=1.0, gamma='scale', class_weight='balanced'))
    ])

    # -----------------------------------------------------------------
    # Step 4: Generate Learning Curve
    '''
    The learning_curve function automatically performs the cross-validation and generates the scores for different training set sizes.
    '''
    # -----------------------------------------------------------------
    print("Generating learning curve...")
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    train_sizes, train_scores, test_scores = learning_curve(
        clf,
        X_tr,
        y_tr,
        cv=cv,
        n_jobs=-1,
        train_sizes=np.linspace(0.1, 1.0, 5),
        scoring='accuracy'
    )

    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)

    # Plot Learning Curve
    plt.figure(figsize=(8, 6))
    plt.plot(train_sizes, train_mean, 'o-', label="Training score")
    plt.plot(train_sizes, test_mean, 'o-', label="Cross-validation score")

    plt.fill_between(
        train_sizes,
        train_mean - train_std,
        train_mean + train_std,
        alpha=0.1
    )
    plt.fill_between(
        train_sizes,
        test_mean - test_std,
        test_mean + test_std,
        alpha=0.1
    )

    plt.title(f"Learning Curve - Subject {sub}")
    plt.xlabel("Training examples")
    plt.ylabel("Accuracy")
    plt.legend(loc="best")
    plt.grid()

    plot_filename = os.path.join(plot_dir, f"LearningCurve_Sub{sub}.png")
    plt.savefig(plot_filename)
    plt.close()
    print(f"   Plot saved to: {plot_filename}")

    # -----------------------------------------------------------------
    # Step 5: Final Training & Model Saving
    # -----------------------------------------------------------------
    clf.fit(X_tr, y_tr) # final training

    model_filename = os.path.join(model_dir, f"Model_Sub{sub}.pkl")
    joblib.dump(clf, model_filename)
    print(f"   Model saved to: {model_filename}")

    # -----------------------------------------------------------------
    # Step 6: Validation Evaluation
    # -----------------------------------------------------------------
    y_pred = clf.predict(X_va) # prediction
    final_acc = accuracy_score(y_va, y_pred)
    final_precision = precision_score(
        y_va, y_pred, average="macro", zero_division=0
    )
    final_f1 = f1_score(
        y_va, y_pred, average="macro", zero_division=0
    )

    print(f"   Validation Accuracy:  {final_acc:.3f}")
    print(f"   Validation Precision: {final_precision:.3f}")
    print(f"   Validation Macro-F1:  {final_f1:.3f}")

    results_list.append({
        'Subject': sub,
        'CV_Mean': np.mean(test_mean),
        'Val_Accuracy': final_acc,
        'Val_Precision': final_precision,
        'Val_F1': final_f1
    })

--- Processing Subject 01 ---
Generating learning curve...
   Plot saved to: /content/drive/MyDrive/FAST-EEG/Results/Plots/LearningCurve_Sub01.png
Computing rank from data with rank=None
    Using tolerance 2.5e+02 (2.2e-16 eps * 64 dim * 1.7e+16  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=2 covariance using EMPIRICAL
Done.
Estimating class=3 covariance using EMPIRICAL
Done.
Estimating class=4 covariance using EMPIRICAL
Done.
   Model saved to: /content/drive/MyDrive/FAST-EEG/Results/Models/Model_Sub01.pkl
   Validation Accuracy:  0.340
   Validation Precision: 0.255
   Validation Macro-F1:  0.282
--- Processing Subject 02 ---
Generating learning curve...
   Plot saved to: /content/drive/MyDrive/FAST-EEG/Results/Plots/LearningCurve_Sub02.png
Computing rank from 

Final report

In [ ]:
# Final Report
if results_list:
  df_results = pd.DataFrame(results_list)

  print("\n\n=== FINAL RESULTS (PER SUBJECT) ===")
  print(df_results)

  # ---- Aggregate statistics across subjects ----
  mean_acc = df_results['Val_Accuracy'].mean()
  std_acc  = df_results['Val_Accuracy'].std()

  mean_prec = df_results['Val_Precision'].mean()
  std_prec  = df_results['Val_Precision'].std()

  mean_f1 = df_results['Val_F1'].mean()
  std_f1  = df_results['Val_F1'].std()

  print("\n=== AGGREGATED RESULTS ACROSS SUBJECTS ===")
  print(f"Accuracy  : {mean_acc:.3f} ± {std_acc:.3f}")
  print(f"Precision : {mean_prec:.3f} ± {std_prec:.3f}")
  print(f"Macro F1  : {mean_f1:.3f} ± {std_f1:.3f}")

  # ---- Save detailed per-subject results ----
  csv_path = os.path.join(base_path, "Results", "final_results_per_subject.csv")
  df_results.to_csv(csv_path, index=False)
  print(f"\nResults CSV saved to: {csv_path}")

else:
  print("\nNo subjects were processed successfully.")



=== FINAL RESULTS (PER SUBJECT) ===
   Subject   CV_Mean  Val_Accuracy  Val_Precision    Val_F1
0       01  0.302000          0.34       0.254687  0.282011
1       02  0.238000          0.32       0.379545  0.323058
2       03  0.337333          0.36       0.374747  0.350677
3       04  0.315333          0.46       0.449206  0.447569
4       05  0.357333          0.38       0.370476  0.358667
5       06  0.219333          0.30       0.260676  0.255566
6       07  0.300667          0.40       0.460995  0.410148
7       08  0.333333          0.42       0.455649  0.411480
8       09  0.310667          0.40       0.390000  0.390000
9       10  0.273333          0.32       0.338681  0.317957
10      11  0.261333          0.44       0.465085  0.426014
11      12  0.278667          0.20       0.177143  0.186667
12      13  0.330667          0.32       0.338100  0.325772
13      14  0.312667          0.28       0.287792  0.268561
14      15  0.288667          0.36       0.394632  0.355259

=

Interpretation of the results:

The low classification performance (accuracy ≈ 0.38, macro-F1 ≈ 0.37) suggests that the CSP + SVM pipeline is not sufficient for decoding imagined speech from EEG. Imagined speech signals are weak, highly variable, and distributed across time and brain regions, making them difficult to capture using variance-based spatial features alone. While CSP is effective for tasks with strong spatial patterns, it may miss important temporal dynamics of imagined speech. In addition, SVM relies on fixed feature representations and cannot model complex temporal dependencies. These results indicate that more expressive models, such as deep neural networks designed to learn joint spatial-temporal features, are better suited for this task.


Run the same pipline but on test set

Extract the labels from the excel sheet

In [ ]:
import h5py
import pandas as pd
import numpy as np

df_labels =  pd.read_excel("/content/drive/MyDrive/FAST-EEG/Test_set/Track3_Answer_Sheet_Test.xlsx", header=None)

for sub in subjects:
    print(f"--- Processing Subject {sub} (TEST) ---")

    try:
        f_te = f"{base_path}/Test_set/Data_Sample{sub}.mat"

        with h5py.File(f_te, "r") as f:
            x_te = f["epo_test"]["x"][:]   # raw array

        # y מהאקסל (50 תוויות)
        sub_int = int(sub)
        y_te = df_labels.iloc[3:53, 2 * sub_int].to_numpy().astype(int) - 1
        n_trials = len(y_te)  # 50

        # ===== FIX: put the axis with 50 trials in axis 0 =====
        # x_te is 3D, one of the axes should be 50 (trials)
        trial_axis = int(np.where(np.array(x_te.shape) == n_trials)[0][0])
        X_te = np.moveaxis(x_te, trial_axis, 0).astype(np.float64)

        # אם את רוצה שזה יהיה בדיוק (trials, channels, time) כמו אצלך,
        # לרוב אחרי moveaxis זה כבר נכון. אם יצא (trials, time, channels) אפשר להפוך:
        if X_te.ndim == 3 and X_te.shape[1] == 795:  # זמן יושב באמצע -> נהפוך
            X_te = np.transpose(X_te, (0, 2, 1))

        if X_te.shape[0] != n_trials:
            raise ValueError(f"Mismatch: X_te has {X_te.shape[0]} trials, y_te has {n_trials} labels")

    except Exception as e:
        print(f"Error loading TEST for Subject {sub}: {e}")
        continue

    print("OK shapes:", X_te.shape, y_te.shape)
    # מכאן את ממשיכה פילטר + מודל

--- Processing Subject 01 (TEST) ---
OK shapes: (50, 64, 795) (50,)
--- Processing Subject 02 (TEST) ---
OK shapes: (50, 64, 795) (50,)
--- Processing Subject 03 (TEST) ---
OK shapes: (50, 64, 795) (50,)
--- Processing Subject 04 (TEST) ---
OK shapes: (50, 64, 795) (50,)
--- Processing Subject 05 (TEST) ---
OK shapes: (50, 64, 795) (50,)
--- Processing Subject 06 (TEST) ---
OK shapes: (50, 64, 795) (50,)
--- Processing Subject 07 (TEST) ---
OK shapes: (50, 64, 795) (50,)
--- Processing Subject 08 (TEST) ---
OK shapes: (50, 64, 795) (50,)
--- Processing Subject 09 (TEST) ---
OK shapes: (50, 64, 795) (50,)
--- Processing Subject 10 (TEST) ---
OK shapes: (50, 64, 795) (50,)
--- Processing Subject 11 (TEST) ---
OK shapes: (50, 64, 795) (50,)
--- Processing Subject 12 (TEST) ---
OK shapes: (50, 64, 795) (50,)
--- Processing Subject 13 (TEST) ---
OK shapes: (50, 64, 795) (50,)
--- Processing Subject 14 (TEST) ---
OK shapes: (50, 64, 795) (50,)
--- Processing Subject 15 (TEST) ---
OK shapes: 

In [ ]:
import os
import numpy as np
import pandas as pd
import scipy.io
import h5py
import matplotlib.pyplot as plt

from mne.filter import filter_data
from mne.decoding import CSP

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, learning_curve
from sklearn.metrics import accuracy_score, precision_score, f1_score
import joblib

# ------------------------------
# Load Excel labels ONCE
# ------------------------------
df_labels = pd.read_excel(
    "/content/drive/MyDrive/FAST-EEG/Test_set/Track3_Answer_Sheet_Test.xlsx",header=None)

# ============================
# Main Loop across all subjects
# ============================
results_list_test = []
for sub in subjects:
    print(f"--- Processing Subject {sub} ---")

    # -----------------------------------------------------------------
    # Step 1: Load TRAIN + VALIDATION + TEST
    # -----------------------------------------------------------------
    try:
        # -------- TRAIN --------
        f_tr = f"{base_path}/Training_set/Data_Sample{sub}.mat"
        d_tr = scipy.io.loadmat(f_tr)
        x_tr = np.asarray(d_tr["epo_train"]["x"])[0, 0]
        y_tr = np.asarray(d_tr["epo_train"]["y"])[0, 0].argmax(0)
        X_tr = np.transpose(x_tr, (2, 1, 0)).astype(np.float64)

        # -------- VALIDATION --------
        f_va = f"{base_path}/Validation_set/Data_Sample{sub}.mat"
        d_va = scipy.io.loadmat(f_va)
        x_va = np.asarray(d_va["epo_validation"]["x"])[0, 0]
        y_va = np.asarray(d_va["epo_validation"]["y"])[0, 0].argmax(0)
        X_va = np.transpose(x_va, (2, 1, 0)).astype(np.float64)

        # -------- TRAINALL = TRAIN + VALIDATION --------
        X_trainall = np.concatenate([X_tr, X_va], axis=0)
        y_trainall = np.concatenate([y_tr, y_va], axis=0)

        # -------- TEST (MATLAB v7.3 -> h5py) --------
        f_te = f"{base_path}/Test_set/Data_Sample{sub}.mat"
        with h5py.File(f_te, "r") as f:
            x_te = f["epo_test"]["x"][:]   # raw shape order differs in test

        # y_test from Excel (FAST style)
        sub_int = int(sub)  # "01" -> 1
        y_te = df_labels.iloc[3:53, 2 * sub_int].to_numpy().astype(int) - 1
        n_trials = len(y_te)  # 50

        # move axis with 50 trials to axis 0 -> (trials, ?, ?)
        trial_axis = int(np.where(np.array(x_te.shape) == n_trials)[0][0])
        X_te = np.moveaxis(x_te, trial_axis, 0).astype(np.float64)

        # ensure (trials, channels, time) i.e. (50,64,795)
        if X_te.ndim == 3 and X_te.shape[1] == 795 and X_te.shape[2] == 64:
            X_te = np.transpose(X_te, (0, 2, 1))

        if X_te.shape[0] != n_trials:
            raise ValueError(f"Mismatch: X_te has {X_te.shape[0]} trials, y_te has {n_trials} labels")

    except FileNotFoundError:
        print(f"File for Subject {sub} not found. Skipping.")
        continue
    except Exception as e:
        print(f"Error loading Subject {sub}: {e}")
        continue

    # -----------------------------------------------------------------
    # Step 2: Filter (4–40 Hz)
    # -----------------------------------------------------------------
    sfreq = 250
    X_trainall = filter_data(X_trainall, sfreq, l_freq=4, h_freq=40, verbose=False)
    X_te = filter_data(X_te, sfreq, l_freq=4, h_freq=40, verbose=False)

    # -----------------------------------------------------------------
    # Step 3: Define Pipeline
    # -----------------------------------------------------------------
    n_components = 8
    clf = Pipeline([
        ('csp', CSP(n_components=n_components, reg=None, log=True, norm_trace=False)),
        ('scaler', StandardScaler()),
        ('svm', SVC(kernel='rbf', C=1.0, gamma='scale', class_weight='balanced'))
    ])

    # -----------------------------------------------------------------
    # Step 4: Learning curve on TRAINALL (optional but consistent)
    # -----------------------------------------------------------------
    print("Generating learning curve (TRAINALL)...")
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    train_sizes, train_scores, cv_scores = learning_curve(
        clf,
        X_trainall,
        y_trainall,
        cv=cv,
        n_jobs=-1,
        train_sizes=np.linspace(0.1, 1.0, 5),
        scoring='accuracy'
    )

    train_mean = np.mean(train_scores, axis=1)
    train_std  = np.std(train_scores, axis=1)
    cv_mean    = np.mean(cv_scores, axis=1)
    cv_std     = np.std(cv_scores, axis=1)

    plt.figure(figsize=(8, 6))
    plt.plot(train_sizes, train_mean, 'o-', label="Training score")
    plt.plot(train_sizes, cv_mean, 'o-', label="Cross-validation score")

    plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1)
    plt.fill_between(train_sizes, cv_mean - cv_std, cv_mean + cv_std, alpha=0.1)

    plt.title(f"Learning Curve - TRAINALL - Subject {sub}")
    plt.xlabel("Training examples")
    plt.ylabel("Accuracy")
    plt.legend(loc="best")
    plt.grid()

    plot_filename = os.path.join(plot_dir, f"LearningCurve_TRAINALL_Sub{sub}.png")
    plt.savefig(plot_filename)
    plt.close()
    print(f"   Plot saved to: {plot_filename}")

    # -----------------------------------------------------------------
    # Step 5: Train on TRAINALL & Save
    # -----------------------------------------------------------------
    clf.fit(X_trainall, y_trainall)
    model_filename = os.path.join(model_dir, f"Model_TRAINALL_Sub{sub}.pkl")
    joblib.dump(clf, model_filename)
    print(f"   Model saved to: {model_filename}")

    # -----------------------------------------------------------------
    # Step 6: Predict on TEST & Evaluate vs Excel labels
    # -----------------------------------------------------------------
    y_pred = clf.predict(X_te)

    test_acc = accuracy_score(y_te, y_pred)
    test_precision = precision_score(y_te, y_pred, average="macro", zero_division=0)
    test_f1 = f1_score(y_te, y_pred, average="macro", zero_division=0)

    print(f"   TEST Accuracy:  {test_acc:.3f}")
    print(f"   TEST Precision: {test_precision:.3f}")
    print(f"   TEST Macro-F1:  {test_f1:.3f}")

    results_list_test.append({
        'Subject': sub,
        'CV_Mean_TRAINALL': float(np.mean(cv_mean)),
        'Test_Accuracy': float(test_acc),
        'Test_Precision': float(test_precision),
        'Test_F1': float(test_f1)
    })

--- Processing Subject 01 ---
Generating learning curve (TRAINALL)...
   Plot saved to: /content/drive/MyDrive/FAST-EEG/Results/Plots/LearningCurve_TRAINALL_Sub01.png
Computing rank from data with rank=None
    Using tolerance 2.7e+02 (2.2e-16 eps * 64 dim * 1.9e+16  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=2 covariance using EMPIRICAL
Done.
Estimating class=3 covariance using EMPIRICAL
Done.
Estimating class=4 covariance using EMPIRICAL
Done.
   Model saved to: /content/drive/MyDrive/FAST-EEG/Results/Models/Model_TRAINALL_Sub01.pkl
   TEST Accuracy:  0.360
   TEST Precision: 0.328
   TEST Macro-F1:  0.302
--- Processing Subject 02 ---
Generating learning curve (TRAINALL)...
   Plot saved to: /content/drive/MyDrive/FAST-EEG/Results/Plots/LearningCurve_TRAINALL

In [ ]:
# ============================
# Final Report (TRAINALL CV + TEST metrics)
# ============================

import os
import pandas as pd

if results_list_test:
    df_results = pd.DataFrame(results_list_test)

    print("\n\n=== FINAL RESULTS (PER SUBJECT) ===")
    desired_cols = [
        "Subject",
        "CV_Mean_TRAINALL",
        "Test_Accuracy",
        "Test_Precision",
        "Test_F1"
    ]
    cols = [c for c in desired_cols if c in df_results.columns] + \
           [c for c in df_results.columns if c not in desired_cols]
    df_results = df_results[cols]
    print(df_results)

    print("\n=== AGGREGATED RESULTS ACROSS SUBJECTS ===")

    # ---- Aggregate TRAINALL CV ----
    if "CV_Mean_TRAINALL" in df_results.columns:
        mean_cv = df_results["CV_Mean_TRAINALL"].mean()
        std_cv  = df_results["CV_Mean_TRAINALL"].std()
        print(f"TRAINALL CV Accuracy (mean across train sizes, per subject): {mean_cv:.3f} ± {std_cv:.3f}")

    # ---- Aggregate TEST ----
    mean_acc = df_results["Test_Accuracy"].mean()
    std_acc  = df_results["Test_Accuracy"].std()

    mean_prec = df_results["Test_Precision"].mean()
    std_prec  = df_results["Test_Precision"].std()

    mean_f1 = df_results["Test_F1"].mean()
    std_f1  = df_results["Test_F1"].std()

    print(f"TEST Accuracy  : {mean_acc:.3f} ± {std_acc:.3f}")
    print(f"TEST Precision : {mean_prec:.3f} ± {std_prec:.3f}")
    print(f"TEST Macro F1  : {mean_f1:.3f} ± {std_f1:.3f}")

    # ---- Save detailed per-subject results ----
    results_dir = os.path.join(base_path, "Results")
    os.makedirs(results_dir, exist_ok=True)

    csv_path = os.path.join(results_dir, "final_results_trainall_test_per_subject.csv")
    df_results.to_csv(csv_path, index=False)
    print(f"\nResults CSV saved to: {csv_path}")

else:
    print("\nNo subjects were processed successfully.")



=== FINAL RESULTS (PER SUBJECT) ===
   Subject  CV_Mean_TRAINALL  Test_Accuracy  Test_Precision   Test_F1
0       01          0.310857           0.36        0.327576  0.301905
1       02          0.255429           0.26        0.253333  0.244697
2       03          0.329143           0.42        0.417820  0.414385
3       04          0.338857           0.32        0.370455  0.324428
4       05          0.346286           0.42        0.438462  0.421228
5       06          0.256000           0.34        0.365022  0.336929
6       07          0.325143           0.46        0.437702  0.435813
7       08          0.331429           0.46        0.472829  0.442429
8       09          0.338286           0.56        0.634975  0.552882
9       10          0.298286           0.30        0.317105  0.287962
10      11          0.282286           0.48        0.531457  0.484197
11      12          0.275429           0.36        0.416868  0.347207
12      13          0.332000           0.28        0